In [1]:
cd ~/imCPA

/mnt/home/icb/alessandro.palma/imCPA


In [2]:
# Import the files 
import tarfile
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import pandas as pd
from PIL import Image
import timeit,time
from plot_utils import Plotter 
import torch
from training_utils import *
import json
import sys

In [3]:
torch.cuda.set_per_process_memory_fraction(1.0, device=None)

In [4]:
config = Config(config_path = '/home/icb/alessandro.palma/imCPA/configs/config_sigma_vae.json') 

In [5]:
t = Trainer(config)

Working on device: cuda
Lodading the data...
Load the data
Successfully loaded the data


In [ ]:
t.train()

Create output directories for the experiment
Beginning training with epochs 100
Running epoch 1


100%|██████████| 401/401 [06:46<00:00,  1.01s/it]

Mean loss after epoch 1: -53261.83017127591
Mean reconstruction loss after epoch 1: -53750.43874444748
Mean kl divergence after epoch 1: -488.6086182855906
Seen batches 0
Rmse = 33.95563888549805
LLH in bit/dim = 1.5509098284444076


Average validation loss: -65427.241286057695
Average validation reconstruction loss: -66525.01802884616
Average kld reconstruction loss: -1097.7767380934495
Seen batches 0
Rmse = 1.6609231233596802
LLH in bit/dim = 1.919506272105305
New best loss is -65427.241286057695
Save new checkpoint at /home/icb/alessandro.palma/checkpoints/vae_resid10_dim100
Running epoch 2


100%|██████████| 401/401 [06:45<00:00,  1.01s/it]

...

Average validation loss: -86801.55649038461
Average validation reconstruction loss: -88394.98016826923
Average kld reconstruction loss: -1593.423832820012
Seen batches 0
Rmse = 1.070376992225647
LLH in bit/dim = 2.550539990564806
New best loss is -86801.55649038461
Save new checkpoint at /home/icb/alessandro.palma/checkpoints/vae_resid10_dim100
Running epoch 16


100%|██████████| 401/401 [06:46<00:00,  1.01s/it]

Mean loss after epoch 16: -88845.96043095387
Mean reconstruction loss after epoch 16: -90470.5588372818
Mean kl divergence after epoch 16: -1624.5982395086503
Seen batches 0
Rmse = 15.83332347869873
LLH in bit/dim = 2.6104285316199927


Average validation loss: -86470.97385817308
Average validation reconstruction loss: -88076.88792067308
Average kld reconstruction loss: -1605.9142456054688
Seen batches 0
Rmse = 1.0772724151611328
LLH in bit/dim = 2.541361788401762
Running epoch 17


  8%|▊         | 34/401 [00:37<06:08,  1.00s/it]

In [47]:
model = t.model.module
checkpoint = torch.load('/home/icb/alessandro.palma/checkpoints/vae_run_res_100')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [48]:
data = next(iter(t.loader_train))

In [ ]:
z = model(data['X'].to(t.device))['z']

In [65]:
torch.exp(-F.gaussian_nll_loss(torch.zeros(512, 512).to(t.device), z, 
                                torch.ones(512).to(t.device), eps=1e-06, reduction='mean'))

tensor(0., device='cuda:0', grad_fn=<ExpBackward>)

In [66]:
z


tensor([[-1.2789, -0.9533, -1.0354,  ..., -2.0077,  1.3127, -0.5311],
        [ 2.5931, -1.5500, -0.3522,  ...,  1.3378,  0.8420,  0.2340],
        [-1.3245, -1.9780, -1.4683,  ...,  1.5472,  0.8240,  1.3067],
        ...,
        [ 0.0954,  0.5935,  2.5115,  ..., -1.1101,  1.4514, -1.8272],
        [-0.8832,  0.3693, -0.8817,  ...,  0.1720, -1.3942,  0.9655],
        [-1.2281,  1.4231, -1.7817,  ...,  0.8463, -0.4898, -1.2811]],
       device='cuda:0', grad_fn=<AddBackward0>)